In [35]:
import _sqlite3
import pandas as pd
import numpy as np

cnx = _sqlite3.connect("corpus.sqlite3")

In [36]:
comments_df = pd.read_sql_query("Select ID_Post, Headline, Body, Status FROM Posts",cnx)
labels_df = pd.read_sql_query("SELECT * From Annotations_consolidated",cnx)

In [37]:
labels = ["ArgumentsUsed","Discriminating","Inappropriate","OffTopic","PersonalStories","PossiblyFeedback","SentimentNegative","SentimentNeutral","SentimentPositive"]

label_map = {y:x for (x,y) in enumerate(labels)}

In [38]:
comments_df

,ID_Post,Headline,Body,Status
0,1,,,deleted
1,2,"Newsletter ""DER STANDARD""",Ich bin begeistert von den STANDARD - Newslett...,online
2,3,Auch begeistert...,... Aber momentan funktioniert das Abmelden od...,deleted
3,4,Abmeldeprobleme,Es ist ganz einfach nervend!\r\nVor kurzem hab...,deleted
4,5,,und sie als mitarbeiter sind natuerlich objektiv,online
...,...,...,...,...
1011768,1011769,None,zwischen der beendigung eines arbeitsverhältni...,online
1011769,1011770,None,du sagst es ja im namen selbst: wegen eindicku...,online
1011770,1011771,None,Was genau haben Sie denn nicht verstanden? Ich...,online
1011771,1011772,None,irgendwie widersprechen Sie sich in Ihrem Post...,online


In [39]:
labels_df.value_counts("ID_Post").isin([9])

ID_Post
432258      True
136446      True
136458      True
136457      True
136456      True
           ...  
17999      False
18001      False
18006      False
18007      False
1010997    False
Length: 11773, dtype: bool

In [40]:
index_values_of_9_label_posts = labels_df.value_counts("ID_Post")[labels_df.value_counts("ID_Post").isin([9])].index
len(index_values_of_9_label_posts)

3599

In [41]:
# Die Kommentare, für die alle 9 Labels annotiert wurden:
comments_filtered_df =comments_df.loc[index_values_of_9_label_posts]


In [42]:
# Anfügen der Spalten für jedes Label in comments_filtered_df
for label in labels:
    comments_filtered_df[label] = np.zeros(len(index_values_of_9_label_posts))

In [43]:
#renaming status to deleted
comments_filtered_df = comments_filtered_df.rename({'Status': 'Deleted'}, axis=1)

In [44]:
# Setzen der Labels in comments_filtered_df
index_values = set(index_values_of_9_label_posts)
for index,row in labels_df.iterrows():
    if row["ID_Post"] in index_values:
        # cur_label = label_map[row["Category"]]
        # cur_label_value = row["Value"]

        comments_filtered_df.at[row["ID_Post"],row["Category"]] = row["Value"]
        comments_filtered_df.loc[comments_filtered_df['Deleted'] == "deleted", 'Deleted'] = 1
        comments_filtered_df.loc[comments_filtered_df['Deleted'] == "online", 'Deleted'] = 0

comments_filtered_df

,ID_Post,Headline,Body,Deleted,ArgumentsUsed,Discriminating,Inappropriate,OffTopic,PersonalStories,PossiblyFeedback,SentimentNegative,SentimentNeutral,SentimentPositive
ID_Post,,,,,,,,,,,,,
432258,432259,None,"Das kann man nur mutmaßen, aber nicht erkennen...",0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
136446,136447,None,Sorry für die verursachte Mühe der Löschungen.,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
136458,136459,None,"Witzig, dass der Poster selber nicht widerspro...",0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
136457,136458,None,Foggwulf und Mouilly haben es Ihnen ja schon e...,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
136456,136457,None,"was auch erklärt, warum körperlich schwächere ...",0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
875899,875900,None,Das beste Mittel gegen die angebliche Hetze si...,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
875937,875938,None,Frankreich und Portugal haben ihnen angeboten ...,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
875900,875901,Video an sehen,die Stinkefinger sind schon ausgefahren.... \r...,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [45]:
#comments_filtered_df['Deleted'] = comments_filtered_df['Deleted'].astype(int)
comments_filtered_df['ArgumentsUsed'] = comments_filtered_df['ArgumentsUsed'].astype(int)
comments_filtered_df['Discriminating'] = comments_filtered_df['Discriminating'].astype(int)
comments_filtered_df['Inappropriate'] = comments_filtered_df['Inappropriate'].astype(int)
comments_filtered_df['OffTopic'] = comments_filtered_df['OffTopic'].astype(int)
comments_filtered_df['PersonalStories'] = comments_filtered_df['PersonalStories'].astype(int)
comments_filtered_df['PossiblyFeedback'] = comments_filtered_df['PossiblyFeedback'].astype(int)
comments_filtered_df['SentimentNegative'] = comments_filtered_df['SentimentNegative'].astype(int)
comments_filtered_df['SentimentNeutral'] = comments_filtered_df['SentimentNeutral'].astype(int)
comments_filtered_df['SentimentPositive'] = comments_filtered_df['SentimentPositive'].astype(int)

In [46]:
comments_filtered_df

,ID_Post,Headline,Body,Deleted,ArgumentsUsed,Discriminating,Inappropriate,OffTopic,PersonalStories,PossiblyFeedback,SentimentNegative,SentimentNeutral,SentimentPositive
ID_Post,,,,,,,,,,,,,
432258,432259,None,"Das kann man nur mutmaßen, aber nicht erkennen...",0,0,0,0,0,0,0,0,1,0
136446,136447,None,Sorry für die verursachte Mühe der Löschungen.,1,0,0,0,0,0,0,1,0,0
136458,136459,None,"Witzig, dass der Poster selber nicht widerspro...",0,0,0,0,1,0,0,0,1,0
136457,136458,None,Foggwulf und Mouilly haben es Ihnen ja schon e...,0,1,0,0,0,0,0,0,1,0
136456,136457,None,"was auch erklärt, warum körperlich schwächere ...",0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
875899,875900,None,Das beste Mittel gegen die angebliche Hetze si...,1,0,0,0,0,0,0,1,0,0
875937,875938,None,Frankreich und Portugal haben ihnen angeboten ...,1,0,0,0,0,0,0,1,0,0
875900,875901,Video an sehen,die Stinkefinger sind schon ausgefahren.... \r...,1,0,0,0,0,0,0,1,0,0


In [47]:
#comments_filtered_df["new_labels"] =  comments_filtered_df["Deleted"].map(str) +", "+comments_filtered_df["ArgumentsUsed"].map(str) +", "+comments_filtered_df["Discriminating"].map(str) +", "+ comments_filtered_df["Inappropriate"].map(str) +", "+ comments_filtered_df["OffTopic"].map(str)+", "+ comments_filtered_df["PersonalStories"].map(str)+", "+ comments_filtered_df["PossiblyFeedback"].map(str)+ ", "+ comments_filtered_df["SentimentNegative"].map(str) + ", " + comments_filtered_df["SentimentNeutral"].map(str) + ", "+ comments_filtered_df["SentimentPositive"].map(str)
#comments_filtered_df["new_labels"] = comments_filtered_df[['Status','ArgumentsUsed','Discriminating','Inappropriate','OffTopic', 'PersonalStories', 'PossiblyFeedback', 'SentimentNegative', 'SentimentNeutral', 'SentimentPositive']].fillna('').agg(' '.join,axis=1)

comments_filtered_df["new_labels"] =  comments_filtered_df["Deleted"].map(str) +comments_filtered_df["ArgumentsUsed"].map(str) +comments_filtered_df["Discriminating"].map(str) + comments_filtered_df["Inappropriate"].map(str) + comments_filtered_df["OffTopic"].map(str)+ comments_filtered_df["PersonalStories"].map(str)+ comments_filtered_df["PossiblyFeedback"].map(str)+  comments_filtered_df["SentimentNegative"].map(str) +  comments_filtered_df["SentimentNeutral"].map(str) + comments_filtered_df["SentimentPositive"].map(str)
#comments_filtered_df["new_labels"] = comments_filtered_df[['Status','ArgumentsUsed','Discriminating','Inappropriate','OffTopic', 'PersonalStories', 'PossiblyFeedback', 'SentimentNegative', 'SentimentNeutral', 'SentimentPositive']].fillna('').agg(' '.join,axis=1)



In [48]:
comments_filtered_df['HeadBody'] = comments_filtered_df['Headline'].astype(str) +'###'+ comments_filtered_df['Body']
comments_filtered_df['HeadBody'] = comments_filtered_df['HeadBody'].replace('###', ' ', regex=True)
comments_filtered_df['HeadBody'] = comments_filtered_df['HeadBody'].replace("None", '', regex=True)

In [49]:
#comments_filtered_df["new_labels"] = comments_filtered_df["new_labels"].astype(str)
comments_filtered_df

,ID_Post,Headline,Body,Deleted,ArgumentsUsed,Discriminating,Inappropriate,OffTopic,PersonalStories,PossiblyFeedback,SentimentNegative,SentimentNeutral,SentimentPositive,new_labels,HeadBody
ID_Post,,,,,,,,,,,,,,,
432258,432259,None,"Das kann man nur mutmaßen, aber nicht erkennen...",0,0,0,0,0,0,0,0,1,0,0000000010,"Das kann man nur mutmaßen, aber nicht erkenne..."
136446,136447,None,Sorry für die verursachte Mühe der Löschungen.,1,0,0,0,0,0,0,1,0,0,1000000100,Sorry für die verursachte Mühe der Löschungen.
136458,136459,None,"Witzig, dass der Poster selber nicht widerspro...",0,0,0,0,1,0,0,0,1,0,0000100010,"Witzig, dass der Poster selber nicht widerspr..."
136457,136458,None,Foggwulf und Mouilly haben es Ihnen ja schon e...,0,1,0,0,0,0,0,0,1,0,0100000010,Foggwulf und Mouilly haben es Ihnen ja schon ...
136456,136457,None,"was auch erklärt, warum körperlich schwächere ...",0,1,0,0,0,0,0,0,1,0,0100000010,"was auch erklärt, warum körperlich schwächere..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875899,875900,None,Das beste Mittel gegen die angebliche Hetze si...,1,0,0,0,0,0,0,1,0,0,1000000100,Das beste Mittel gegen die angebliche Hetze s...
875937,875938,None,Frankreich und Portugal haben ihnen angeboten ...,1,0,0,0,0,0,0,1,0,0,1000000100,Frankreich und Portugal haben ihnen angeboten...
875900,875901,Video an sehen,die Stinkefinger sind schon ausgefahren.... \r...,1,0,0,0,0,0,0,1,0,0,1000000100,Video an sehen die Stinkefinger sind schon aus...


In [50]:
display(comments_filtered_df.dtypes)

ID_Post               int64
Headline             object
Body                 object
Deleted              object
ArgumentsUsed         int32
Discriminating        int32
Inappropriate         int32
OffTopic              int32
PersonalStories       int32
PossiblyFeedback      int32
SentimentNegative     int32
SentimentNeutral      int32
SentimentPositive     int32
new_labels           object
HeadBody             object
dtype: object

In [17]:
#Drop the ID_Post-column
comments_filtered_df=comments_filtered_df.drop("ID_Post",axis=1)

In [51]:
comments_filtered_df.to_csv('df_numlabel.csv',sep="\t")

In [19]:
#Verteilung der Label-Werte nach Label:
for label in labels:
    print(comments_filtered_df.value_counts(label))

ArgumentsUsed
0    2577
1    1022
dtype: int64
Discriminating
0    3317
1     282
dtype: int64
Inappropriate
0    3296
1     303
dtype: int64
OffTopic
0    3019
1     580
dtype: int64
PersonalStories
0    3552
1      47
dtype: int64
PossiblyFeedback
0    3527
1      72
dtype: int64
SentimentNegative
0    1908
1    1691
dtype: int64
SentimentNeutral
1    1865
0    1734
dtype: int64
SentimentPositive
0    3556
1      43
dtype: int64


In [20]:
correlation_matrix = np.zeros((len(labels),len(labels)))

for label1 in labels:
    for label2 in labels:
        if (label1 == label2):
            continue
        two_argument_view = comments_filtered_df[[label1,label2]]

        sum = 0
        for index, row in two_argument_view.iterrows():
            sum += row[label1]*row[label2]

        avgl1l2 = sum/comments_filtered_df.shape[0]


        sum = 0
        for index, row in comments_filtered_df[[label1]].iterrows():
            sum += row[label1]

        avgl1 = sum/comments_filtered_df.shape[0]


        sum = 0
        for index, row in comments_filtered_df[[label2]].iterrows():
            sum += row[label2]

        avgl2 = sum/comments_filtered_df.shape[0]

        corl1l2 = avgl1l2-avgl1*avgl2
        jkl = ((label_map[label1],label_map[label2]),corl1l2)
        correlation_matrix.itemset((label_map[label1],label_map[label2]),corl1l2)

# Covariance matrix of the 9 features:
correlation_matrix+np.diag(np.ones(9))


array([[ 1.00000000e+00, -1.39146737e-02, -1.14037883e-02,
        -2.77025023e-02,  4.34940674e-03, -1.51310902e-03,
        -6.22921637e-02,  6.34621037e-02, -1.16994000e-03],
       [-1.39146737e-02,  1.00000000e+00,  2.57249378e-03,
        -4.84744574e-03, -1.02325358e-03, -1.28968244e-03,
         3.95947564e-02, -3.86585882e-02, -9.36168169e-04],
       [-1.14037883e-02,  2.57249378e-03,  1.00000000e+00,
         1.14392246e-02, -1.09945332e-03,  1.37213565e-03,
         4.18545765e-02, -4.08486937e-02, -1.00588282e-03],
       [-2.77025023e-02, -4.84744574e-03,  1.14392246e-02,
         1.00000000e+00, -1.27099922e-03,  2.05523114e-03,
         1.37497673e-02, -1.21021700e-02, -1.64759730e-03],
       [ 4.34940674e-03, -1.02325358e-03, -1.09945332e-03,
        -1.27099922e-03,  1.00000000e+00, -2.61256233e-04,
        -5.30232804e-03,  4.06908127e-03,  1.23324677e-03],
       [-1.51310902e-03, -1.28968244e-03,  1.37213565e-03,
         2.05523114e-03, -2.61256233e-04,  1.000000

In [21]:
# We see that the labels are nearly uncorrelated, with exception of the label pair (6,7) [SentimentNegative,SentimentNeutral], 
#which however goes to show that the definition of what exactly makes a comment be of negative sentiment is rather unclear.

# The labels corresponding with the numbers of the matrix:
label_map

{'ArgumentsUsed': 0,
 'Discriminating': 1,
 'Inappropriate': 2,
 'OffTopic': 3,
 'PersonalStories': 4,
 'PossiblyFeedback': 5,
 'SentimentNegative': 6,
 'SentimentNeutral': 7,
 'SentimentPositive': 8}

In [22]:
# two_argument_view = comments_filtered_df[[label1,label2]]
# index, row = next(comments_filtered_df[[label1]].iterrows())
#
# row[label2]

In [23]:
cf_df = comments_filtered_df
cf_df2 = cf_df
for index in cf_df2:
    if cf_df2['Body'] is None:
        cf_df2.drop(index, axis=1)


In [24]:
cf_df2['Body']
for index in cf_df2:
    if cf_df2['Body'] is None:
        print(cf_df)
    else:
        print("no none")
len(cf_df2)

cf_df2.to_csv('df2.csv',sep="\t")

no none
no none
no none
no none
no none
no none
no none
no none
no none
no none
no none
no none
no none
no none


In [25]:
for row in cf_df2['Body']:
    if row is None:
        row = cf_df2['Headline']

In [26]:
for index in cf_df2:
    if cf_df2['Body'] is None:
        print(cf_df)
    else:
        print("no none")

no none
no none
no none
no none
no none
no none
no none
no none
no none
no none
no none
no none
no none
no none


In [27]:
cf_df2.to_csv('df222.csv',sep="\t")

In [28]:
cf_df3 = cf_df2
cf_df3['HeadBody'] = cf_df3['Headline'].astype(str) +'###'+ cf_df3['Body']

In [29]:
tdf = cf_df3

In [30]:
tdf['HeadBody'] = tdf['HeadBody'].replace('###', ' ', regex=True)
tdf

,Headline,Body,Deleted,ArgumentsUsed,Discriminating,Inappropriate,OffTopic,PersonalStories,PossiblyFeedback,SentimentNegative,SentimentNeutral,SentimentPositive,new_labels,HeadBody
ID_Post,,,,,,,,,,,,,,
432258,None,"Das kann man nur mutmaßen, aber nicht erkennen...",0,0,0,0,0,0,0,0,1,0,0000000010,"None Das kann man nur mutmaßen, aber nicht erk..."
136446,None,Sorry für die verursachte Mühe der Löschungen.,1,0,0,0,0,0,0,1,0,0,1000000100,None Sorry für die verursachte Mühe der Löschu...
136458,None,"Witzig, dass der Poster selber nicht widerspro...",0,0,0,0,1,0,0,0,1,0,0000100010,"None Witzig, dass der Poster selber nicht wide..."
136457,None,Foggwulf und Mouilly haben es Ihnen ja schon e...,0,1,0,0,0,0,0,0,1,0,0100000010,None Foggwulf und Mouilly haben es Ihnen ja sc...
136456,None,"was auch erklärt, warum körperlich schwächere ...",0,1,0,0,0,0,0,0,1,0,0100000010,"None was auch erklärt, warum körperlich schwäc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875899,None,Das beste Mittel gegen die angebliche Hetze si...,1,0,0,0,0,0,0,1,0,0,1000000100,None Das beste Mittel gegen die angebliche Het...
875937,None,Frankreich und Portugal haben ihnen angeboten ...,1,0,0,0,0,0,0,1,0,0,1000000100,None Frankreich und Portugal haben ihnen angeb...
875900,Video an sehen,die Stinkefinger sind schon ausgefahren.... \r...,1,0,0,0,0,0,0,1,0,0,1000000100,Video an sehen die Stinkefinger sind schon aus...


In [31]:
tdf.to_csv('tdf_testdataframe.csv',sep="\t")

BELOW HERE PROB USELESS




In [32]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import naive_bayes

In [33]:
docs = tdf['HeadBody'].tolist()
count_vectorizer = CountVectorizer()

In [ ]:
for row in tdf['HeadBody']:
    if tdf['HeadBody'] is NaN:
        print(row)

In [ ]:
text_train, text_test, label_train, label_test = train_test_split(tdf['HeadBody'], tdf['SentimentNegative'], 
                                                                  test_size=0.30, 
                                                                  random_state=1234, shuffle=True)

In [ ]:
polarity_count_vectorizer = CountVectorizer()
polarity_bow_matrix = polarity_count_vectorizer.fit_transform(text_train.values.astype('U'))

In [ ]:
svm_classifier = svm.LinearSVC()

In [ ]:
svm_classifier.fit(polarity_bow_matrix, label_train)

In [ ]:
polarity_bow_matrix_test = polarity_count_vectorizer.transform(text_test.values.astype('U'))
test = svm_classifier.predict(polarity_bow_matrix_test)

In [ ]:
correct_answers = np.sum(np.equal(test, label_test))
accuracy = correct_answers / (len(test)*1.0) * 100
print(accuracy)